# The Littlest JupyterHub による軽量Python実習環境の構築

[JupyterHub](https://jupyter.org/hub) は、Webブラウザからアクセス可能なマルチユーザ対応の認証機能付きJupyterNotebookサーバです。

JupyterHubを利用して管理者が用意したNotebookをユーザがブラウザからすぐに実行可能な環境を提供できるため、Pythonによるプログラミング研修やワークショップを開催したり、講義演習環境として活用したりするのに適しています。

本テンプレートでは、JupyterHubを小規模なグループで手軽に利用することを想定し、単一のサーバで実行するために開発された「[The Littlest JupyterHub](https://tljh.jupyter.org/)」（以下 “TLJH” と略）をVCPを用いて構築します。  

構築したTLJH環境の操作はWebブラウザ経由で行うことになります。構築するVCノードにブラウザからアクセスできるように事前にネットワークを設定してください。直接アクセスできない場合は、例えばリバースプロキシを使う、DNATで制御する、などの手段があります。

![](images/overview.png)

## 構築環境情報の入力
TLJH環境の構築情報を入力します。必要に応じ、下記の情報を修正してください。


In [ ]:
# UnitGroup名
ugroup_name = 'TLJH_template'

# プロバイダ
vc_provider = 'aws'

# フレーバー
vcnode_flavor = 'small'

# VCノードに付与する固定IPアドレス
fixed_ipaddress = '172.30.2.201'

# VCノードのディスクサイズ (単位: GB)
volume_size = 32

# TLJH管理者ユーザ名とパスワードの指定
admin_user = 'admin-user01'
admin_password = 'admin-user01'

## VCノードの作成

はじめに、アプリケーションVCP を利用するために必要なアクセストークンを入力し、VCP SDK を初期化します。

### アクセストークンの入力

In [ ]:
from getpass import getpass
vcc_access_token = getpass()

### VCP の初期化
VCP を初期化します。エラーになった場合、この章のセルを `unfreeze` してから、もう一度アクセストークンを入力してください。

In [ ]:
from common import logsetting
from vcpsdk.vcpsdk import VcpSDK

# VCP SDKの初期化
vcp = VcpSDK(vcc_access_token)

### TLJH を実行するVCノードの作成

クラウド上のインスタンスをVCノードとして作成します。

 ### VCノードのspecを指定する
 
TLJH を利用するのに十分な性能・容量のノードspecを指定します。  

In [ ]:
# UnitGroup の作成
unit_group = vcp.create_ugroup(ugroup_name)

# VCノード spec
spec = vcp.get_spec(vc_provider, vcnode_flavor)

# spec オプション (ディスクサイズ 単位:GB)
spec.volume_size = volume_size

# spec オプション (固定割当IPアドレス)
spec.ip_addresses = [fixed_ipaddress]

# ssh keyfiles
import os
ssh_public_key = os.path.expanduser('~/.ssh/id_rsa.pub')
spec.set_ssh_pubkey(ssh_public_key)

# BaseコンテナとしてTLJHインストール済みのイメージを指定
spec.image = 'harbor.vcloud.nii.ac.jp/vcp/tljh:20230327'
spec.params_v = ['/sys/fs/cgroup:/sys/fs/cgroup:ro',
                 '/lib/modules:/lib/modules:ro',
                 '/var/lib/docker']


### Unitの作成とVCノードの起動
Unitを作成します。Unitを作成すると同時に VCノード（ここでは Amazon EC2インスタンス）が起動します。処理が完了するまで1分半～2分程度かかります。

In [ ]:
# Unitの作成（同時に VCノードが作成される）
unit = unit_group.create_unit('tljh-node', spec)

### 疎通確認

まず、ssh の `known_hosts` の設定を行います。

その後、VCノードに対して`uname -a`を実行し、`ubuntu x86_64 Linux` が起動していることを確認します。起動していない場合は、`spec.image` に誤りがあります。本Notebook下部にある「環境の削除」を実行、 `spec.image` を修正、全てのセルを `unfreeze` してから、最初から再実行してください。

In [ ]:
# unit_group.find_ip_addresses() は UnitGroup内の全VCノードのIPアドレスのリストを返します
ip_address = unit_group.find_ip_addresses(node_state='RUNNING')[0] # 今は１つのVCノードのみ起動しているので [0] で最初の要素を取り出す
print(ip_address)

# ssh 設定
!touch ~/.ssh/known_hosts
!ssh-keygen -R {ip_address}    # ~/.ssh/known_hosts から古いホストキーを削除する
!ssh-keyscan -H {ip_address} >> ~/.ssh/known_hosts    # ホストキーの登録
# システムの確認
!ssh -l vcp {ip_address} uname -a

## TLJH (The Littlest JupyterHub) 環境の構築

### TLJHの起動

- 管理者ユーザ名とパスワードを指定して起動します。
- TLJHコンテナ上で `tljh-config` コマンドを実行することでユーザ追加だけでなく他の様々な設定を行うことができます。
- cf. <https://tljh.jupyter.org/en/latest/topic/tljh-config.html>

In [ ]:
# TLJHの起動
!ssh -l vcp {ip_address} sudo /opt/tljh/hub/bin/python3 /opt/tljh/src/bootstrap/bootstrap.py --admin {admin_user}:{admin_password}


#### TLJH の 起動確認
`jupyterhub.service`が`active` になっていることを確認します。
以下のセルを実行します。

In [ ]:
!ssh -l vcp {ip_address} sudo systemctl status jupyterhub

#### TLJH の Web インターフェースのカスタマイズ

今、このノートブックを実行しているJupyterNotebook環境と、構築したTLJHのJupyterHubのWebインターフェースを識別しやすくするために、TLJH側にカスタマイズしたCSSファイルを設定します。

以下のセルを実行します。

In [ ]:
!scp ./files/custom.css  vcp@{ip_address}:/tmp/
!ssh -l vcp {ip_address} sudo mkdir -p /etc/skel/.jupyter/custom
!ssh -l vcp {ip_address} sudo cp /tmp/custom.css /etc/skel/.jupyter/custom/custom.css

## TLJH の管理と利用

VCノード上に起動したTLJHにブラウザからアクセスし、Python実習環境として必要な作業を行います。

### 管理者ユーザとしてブラウザからログイン

次のセルとを実行するとログイン先URLを作成します。
直接VCノードのIPアドレスでアクセスできない場合は、環境に合わせて変更してください。

In [ ]:
print("https://{}/".format(ip_address))

#### ログイン

指定した管理者名とパスワードを入力しログインしてください。
管理者名とパスワードは冒頭のTLJH環境の構築情報として設定しています。
![](images/tljh-login.png)

#### 初期画面

ログイン後の初期画面は、ファイルやディレクトリが1つもない状態となっています。

![](images/tljh-filebrowser.png)

### 一般ユーザ (student) アカウントの追加

管理者用コントロールパネルの機能を利用し、管理者以外のユーザを追加します。

#### 画面右上の **Control Panel** ボタンを押す

![](images/tljh-control_1.png)

#### **Admin** メニューを選択する

![](images/tljh-control_2.png)

#### **Add Users** ボタンを押す

![](images/tljh-control_3.png)

#### アカウント名を入力して **Add Users** ボタンを押す

- アカウント名は改行して複数入力することが可能

![](images/tljh-addusers.png)

### Python パッケージの追加

Python実習環境として利用するために、ライブラリ・パッケージの追加を試します。

- cf. <https://tljh.jupyter.org/en/latest/howto/env/user-environment.html#installing-pip-packages>

<p>
    
ここでは以下の [pip](https://pypi.org/project/pip/) パッケージをインストールします。

- [numpy](https://numpy.org/)
- [matplotlib](https://matplotlib.org/)
- [pandas](https://pandas.pydata.org/)


#### 管理者ユーザとしてログインしてTerminalを開く

![](images/tljh-terminal.png)

#### Terminalで以下のコマンドを実行する

```
sudo -E pip --no-cache-dir install numpy matplotlib pandas
```

Terminal を使用せずに、以下のセルでも同じコマンド操作が可能です。

In [ ]:
!ssh -l vcp {ip_address} sudo -E /opt/tljh/user/bin/pip --no-cache-dir install numpy matplotlib pandas

#### インストールできたことを確認する

```
sudo -E pip --no-cache-dir list | grep -e numpy -e matplotlib -e pandas
```

  - 正常にインストールできていれば、以下のようにバージョン情報が出力される。

    ```
    matplotlib             3.7.1
    matplotlib-inline      0.1.6
    numpy                  1.24.2
    pandas                 1.5.3
    ```

Terminal を使用せずに、以下のセルでも同じコマンド操作が可能です。

In [ ]:
!ssh -l vcp {ip_address} sudo -E /opt/tljh/user/bin/pip --no-cache-dir list | grep -e numpy -e matplotlib -e pandas

### 教材Notebookの配布（ファイル共有）

実習用に用意された教材がある場合、TLJH環境の各ユーザから読み込み専用の共有フォルダにアクセスさせることで教材配布が行えるようにします。

- cf. <https://tljh.jupyter.org/en/latest/howto/content/share-data.html#option-2-create-a-read-only-shared-folder-for-data>

#### 管理者ユーザのTerminalで共有フォルダを作成する

- 管理者ユーザとしてログインし、Terminal からコマンドを実行して共有フォルダを作成します。
- 共有フォルダは `/srv/data/` 以下であれば任意のフォルダ名が使用できます。

```
sudo mkdir -p /srv/data/shared
```

Terminal を使用せずに、以下のセルでも同じコマンド操作が可能です。

In [ ]:
!ssh -l vcp {ip_address} sudo mkdir -p /srv/data/shared

#### ユーザのホームディレクトリに共有フォルダへのリンクを作成する

- `/etc/skel/` ディレクトリに共有フォルダへのシンボリックリンクを作成します。  
  これにより、新規に追加したユーザのホームディレクトリに共有フォルダが表示されます。
  * ユーザ追加後、一度もログインしたことがないユーザは初回ログイン時に共有フォルダが表示されます。
- 管理者ユーザとしてログインし、 Terminalから以下のコマンドを実行します。

```
sudo ln -s /srv/data/shared /etc/skel/
```

Terminal を使用せずに、以下のセルでも同じコマンド操作が可能です。

In [ ]:
!ssh -l vcp {ip_address} sudo ln -sf /srv/data/shared /etc/skel/

#### TLJH環境の共有フォルダにファイルを配置する

サンプルのNotebookファイル [sample-notebook.ipynb](./files/sample-notebook.ipynb) を用意したので、これをJupyterHubにコピーします。

以下のセルを実行することで、ここからVCノードを経由してTLJHコンテナ上の共有フォルダにコピーを行います。

In [ ]:
filename = 'sample-notebook.ipynb'

# このNotebookからVCノードのtmpへのコピー
!scp ./files/{filename} vcp@{ip_address}:/tmp/{filename}

# VCノード上のtmpから共有フォルダへのコピー
!ssh -l vcp {ip_address} sudo cp /tmp/{filename} /srv/data/shared/{filename}

### 一般ユーザとしてログインし、共有フォルダを開く

一般ユーザとしてTLJHにログインすると、ホームディレクトリに共有フォルダが表示されます。

> 管理者ユーザとしてログイン中の場合、一度ログアウトする必要があります。

## (参考) 応用編

ここでは、TLJH の運用環境をさらにカスタマイズするためのいくつかの方法を紹介します。

### Jupyter Notebook 拡張機能の導入

そのままでも十分使いやすいJupyter Notebookですが、 `jupyter_contrib_nbextensions` というライブラリを導入することで様々な拡張機能が使えるようになり、より便利になります。

- cf. <https://tljh.jupyter.org/en/latest/howto/admin/enable-extensions.html>

#### 管理者ユーザのTerminalでコマンド実行

管理者ユーザとしてログインし、Terminal から以下のコマンドを実行します。

1.  pip パッケージをインストールする

```
sudo -E pip --no-cache-dir install jupyter_contrib_nbextensions==0.5.1
```

2. 拡張機能のスタイルファイルをJupyter環境に追加する

```
sudo -E jupyter contrib nbextension install --sys-prefix
```

3. [Table of Contents (2)](https://jupyter-contrib-nbextensions.readthedocs.io/en/latest/nbextensions/toc2/README.html) （目次表示） の拡張機能を有効化する

```
sudo -E jupyter nbextension enable toc2/main --sys-prefix
```

4. 現在有効な拡張機能を確認する

```
jupyter nbextension list
```

Terminal を使用せずに、以下のセルでも同じコマンド操作が可能です。

In [ ]:
# パッケージのインストール
!ssh -l vcp {ip_address} sudo -E /opt/tljh/user/bin/pip --no-cache-dir install jupyter_contrib_nbextensions==0.5.1

# 拡張機能の追加
!ssh -l vcp {ip_address} sudo -E /opt/tljh/user/bin/jupyter contrib nbextension install --sys-prefix

# 拡張機能の有効化
!ssh -l vcp {ip_address} sudo -E /opt/tljh/user/bin/jupyter nbextension enable toc2/main --sys-prefix

# 現在有効な拡張機能の確認
!ssh -l vcp {ip_address} /opt/tljh/user/bin/jupyter nbextension list

### 各ユーザのNotebookサーバのリソース制限

各ユーザが使用できる最大メモリ量やCPUを管理者が `tljh-config` コマンドで設定して制限することができます。

- cf. <https://tljh.jupyter.org/en/latest/topic/tljh-config.html#user-server-limits>

#### メモリ制限

個々のユーザが使用可能な最大メモリ量を指定します。  
管理者ユーザとしてログインし、Terminal から以下のコマンドを実行します。

```
sudo tljh-config set limits.memory 1G
```

- `None` を指定すると、メモリ制限は無効化される。

#### CPU制限

各ユーザが使用可能なCPUコアの合計を指定します。  
管理者ユーザとしてログインし、Terminal から以下のコマンドを実行します。

```
sudo tljh-config set limits.cpu 2
```

- `2` の場合、2個のCPUをフルに使用できることを表す。 `0.5` は、1個のCPUの半分を表す。

#### リソース制限設定の適用

実行中のTLJH環境に設定を反映させるためには、管理者ユーザのTerminalから以下のコマンドを実行します。

1. 確認

```
sudo tljh-config show
```

2. 適用

```
sudo tljh-config reload
```

## 環境の削除

ここまで作成した全てのリソース（UnitGroup, Unit、VCノード）を削除します。  
この操作を行うことで AWS EC2インスタンスやAzure VMなどのクラウドに作成したリソースが削除されます。

In [ ]:
unit_group.cleanup()

削除後の状態を確認します。

In [ ]:
# UnitGroupの一覧を DataFrame で表示する
vcp.df_ugroups()

In [ ]:
# UnitGroup強制削除
# UnitGroup作成後、エラーが発生するなど強制的に削除する必要が生じた場合のみ、コメントを外して利用します。
# ugroup = vcp.get_ugroup('TLJH_template')
# ugroup.cleanup()